In [1]:
from PyPDF2 import PdfReader
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [16]:
import fitz  # import package PyMuPDF

# Open some document, for example a PDF (could also be EPUB, XPS, etc.)
doc = fitz.open("jio.pdf")

# Load a desired page. This works via 0-based numbers
page = doc[2]  # this is the first page

# Look for tables on this page 
# display the table count
tabs = page.find_tables()
print(f"{len(tabs.tables)} table(s) on {page}")

0 table(s) on page 2 of jio.pdf


In [3]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings

In [4]:
text = ""
for page_num in range(doc.page_count):
        page = doc.load_page(page_num)
        text += page.get_text("text")
text=re.sub(r'\n', ' ', text)

In [ ]:
# with open("tcs_annual.pdf", 'rb') as file:
# # with open("drive/MyDrive/Resumes/Frontend Developer Role.pdf", 'rb') as file:
#     # Create a PdfReader object
#     pdf_reader = PdfReader(file)
#     # Access PDF properties and content using pdf_reader object
#     num_pages = len(pdf_reader.pages)
#     print(f"Number of pages in the PDF: {num_pages}")

#     text = ""
#     for page in pdf_reader.pages:
#             text += ' '+page.extract_text()
#     text=re.sub(r'\n', ' ', text)

# #     text_splitter = RecursiveCharacterTextSplitter(
# #             chunk_size=1000,
# #             chunk_overlap=200,
# #             length_function=len
# #             )
#     text_splitter = SemanticChunker(HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))

#     chunks = text_splitter.split_text(text=text)


In [5]:
text_splitter = SemanticChunker(HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2'))
chunks = text_splitter.split_text(text=text)

C:\Users\Mukesh\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
text_Rec_splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=200,
            length_function=len
            )

In [12]:
Recchunks = text_Rec_splitter.split_text(text=text)

In [13]:
len(Recchunks)

1342

In [6]:
min_index = chunks.index(min(chunks, key=len))

# Find the index of the largest string
max_index = chunks.index(max(chunks, key=len))

print(f"Index of the smallest string: {min_index}")
print(f"Index of the largest string: {max_index}")

Index of the smallest string: 50
Index of the largest string: 214


In [14]:
len(chunks[214])

52327

In [40]:
print(text)

329 Integrated Annual Report 2022-23Sole Sourced  ContractNon-competitive agreements that allow a single vendor to fulfill the needs of the contractual requirements. These types of contracts can be won when the competitor set narrows down significantly and comes down to a single vendor discussion, given the nature of the client’s solution requirements. Special Economic ZoneIn India, these are designated areas in which business and trade laws are different from the rest of the country, with various benefits and tax breaks to promote exports, attract investments, and create local jobs. STEM An acronym for education in the fields of science, technology, engineering and math. T&M See Time and Materials Contract TCFD Acronym for Task Force on Climate-related Financial Disclosures tCO2e Acronym for tonnes of carbon dioxide equivalent TCS Pace™ A brand promise that represents the way TCS channels its domain knowledge and organizational units – business and technology services, industry soluti

In [6]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


In [29]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

In [30]:
VectorStore = FAISS.from_texts(chunks, embedding=embeddings)

In [27]:
import pickle

In [25]:
doc = VectorStore.similarity_search("Who is CEO of TCS")

In [26]:
doc

[Document(page_content='supplier category. TCS was recognized for consistently delivering  new capabilities with automation solutions that help improve accuracy for Microsoft, while also increasing its speed to market. TCS was cited for its expertise across industries that enables Microsoft to better serve customers, showcasing what it means to focus on strategy, impact, agility, and modernization.Tata Group Chairperson and TCS Chairman, N Chandrasekaran ,  was bestowed with the Eisenhower Global Leadership Award  for his contribution to commerce, by the non-profit, Business Council for International Understanding. Hans Vestberg, the Chairman and CEO of Verizon, presented N Chandrasekaran with the award and described him as “one of the most  purpose-driven and influential leaders of our time”. Selected by Sainsbury’s, a leading supermarket retailer in UK,  as its transformation partner to modernize its infrastructure landscape using TCS Enterprise Cloud™. Additionally, TCS will  provid

In [8]:
# from langchain.llms import CTransformers

In [11]:
# config = {'max_new_tokens': 512, 'repetition_penalty': 1.1, 'context_length': 2000, 'temperature':0.1,"gpu_layers":15 }
# llm = CTransformers(model = "llama-2-7b-chat.Q2_K.gguf", model_type = "llama", config=config, gpu_layers=15)

In [9]:
from langchain_community.llms import LlamaCpp

In [ ]:
n_gpu_layers = 15 # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
llm = LlamaCpp(
    model_path="llama-2-7b-chat.Q2_K.gguf",
    f16_kv=True,  # MUST set to True, otherwise you will run into problem after a couple of calls
    verbose=True,  # Verbose is required to pass to the callback manager
    n_ctx=2000,
    n_threads=5
)

In [ ]:
# n_gpu_layers = 20 # The number of layers to put on the GPU. The rest will be on the CPU. If you don't know how many layers there are, you can use -1 to move all to GPU.
# llm = LlamaCpp(
#     model_path="llama-2-7b-chat.Q2_K.gguf",
#     n_gpu_layers=n_gpu_layers,
#     n_ctx=2000,
# )

In [12]:
from accelerate import Accelerator
accelerator = Accelerator()

In [13]:
llm = accelerator.prepare(llm)

In [14]:
from langchain.chains import ConversationalRetrievalChain
chain = ConversationalRetrievalChain.from_llm(llm, VectorStore.as_retriever(), return_source_documents=True)

In [15]:
chat_history = []

query = "financiacl gain in 2023"
result = chain.invoke({"question": query, "chat_history": chat_history})


print(result['answer'])

In [2]:
!set REPLICATE_API_TOKEN=r8_YXWL8QgUHS37ia4m2AExakBnk5PgdYu3MP4Fo

In [1]:
import os

# from decouple import config
from langchain_community.llms import Replicate

# REPLICATE_API_TOKEN = config("REPLICATE_API_TOKEN")
# os.environ["REPLICATE_API_TOKEN"] = REPLICATE_API_TOKEN
os.environ["REPLICATE_API_TOKEN"] = "r8_YXWL8QgUHS37ia4m2AExakBnk5PgdYu3MP4Fo"



llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    model_kwargs={"temperature": 0.75, "max_length": 500, "top_p": 1},
)
prompt = """
User: Explain the following question step by step. mahatma gandhi?
Assistant:
"""
# llm(prompt)


response: str = llm(prompt)

print(response)

C:\Users\Mukesh\AppData\Roaming\Python\Python310\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Sure! Mahatma Gandhi was a political and spiritual leader in India who played a significant role in the country's struggle for independence from British colonial rule. Here's a step-by-step explanation of his life and legacy:

Step 1: Early Life and Education
Mahatma Gandhi was born on October 2, 1869, in Porbandar, a coastal town in the state of Gujarat, India. His father, Karamchand Gandhi, was a local administrator, and his mother, Putlibai, was a deeply religious woman who had a significant influence on Gandhi's spiritual development. Gandhi was trained as a lawyer and later moved to South Africa, where he experienced discrimination and racism firsthand.

Step 2: South African Experience and Development of Satyagraha
In 1893, Gandhi moved to South Africa to practice law. He was shocked by the racism and discrimination he encountered, and this experience had a profound impact on his beliefs and political views. He developed the concept of Satyagraha (nonviolent resistance), which be

In [34]:
from langchain_community.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import pickle

In [48]:
with open(f"tcs_annual.pkl", "rb") as f:
        VectorStore = pickle.load(f)
# llm = Replicate(
#     model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
#     model_kwargs={"temperature": 0.75, "max_length": 500},
# )
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
llm = Replicate(
    model="a16z-infra/llama13b-v2-chat:df7690f1994d94e96ad9d568eac121aecf50684a0b0963b25a41cc40061269e5",
    model_kwargs={"temperature": 0.75, "max_length": 3000,  "stream":True},
    callback_manager=callback_manager,
   
)    
# prompt = """
# User: Explain the following question step by step. mahatma gandhi?
# Assistant:
# """
# response: str = llm(prompt=prompt)

# print(response)
chain = ConversationalRetrievalChain.from_llm(llm, VectorStore.as_retriever(), return_source_documents=True)

chat_history = []

query = "Describe letter of CEO to stalkeholder"
result = chain.invoke({"question": query, "chat_history": chat_history})


In [46]:
prompt = """
User: Explain the following question step by step. mahatma gandhi?
Assistant:
"""
response: str = llm(prompt=prompt)


In [6]:
result

{'question': 'Who is ceo of tcs',
 'chat_history': [],
 'answer': '',
 'source_documents': [Document(page_content='initiatives that favor a few strategic partners with end-to-end service offerings, the right innovation capabilities and scale. All these trends play to our strengths. Our scale, structure, and  ability to bring together different capabilities into a seamless service delivery team, helped us win several large deals throughout the year.  Letter to Shareholders  |  9 So long, farewell This year marks a key milestone in my journey with TCS, and  an important transition point for the company, as I step down from my role as CEO and MD on 31st May 2023.  It has been an absolute privilege and an unparalleled learning experience to lead our company in the last six years, a period of tremendous growth and transformation for us.   On a more personal note, I am grateful to Chandra, our  Chairman, for his mentorship and backing over the years, to our Board members for their guidance, 